In [ ]:
import cv2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


video1 =cv2.VideoCapture(0)
video2 =cv2.VideoCapture(1)

while True:
    ret0,frame0 = video1.read()
    ret1,frame1 = video2.read()

    if (ret0):
        cv2.imshow('Camera 0',frame0)

    if (ret1):
        cv2.imshow('Camera 1',frame1)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video1.release()
video2.release()

cv2.destroyAllWindows()

In [4]:

import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import *


model=YOLO('yolov8s.pt')
#signal counter
def signal_counter(count_car):
    if count_car == 0:
        pass
    else:
        number_of_cars = count_car
        mean_time_headway = 30/number_of_cars
        flow_rate = 3600/mean_time_headway
        Amber = 2
 

        approach_width = 4

        saturation_flow = 525*approach_width

        Y_ratio = saturation_flow/flow_rate


        critical_Y_ratio = Y_ratio

        number_of_phases = 2

        time_loss = 2

        all_red = 4

        loss_time_per_cycle = number_of_phases*time_loss + all_red


        Total_cycle_length = (1.5*loss_time_per_cycle + 5)/(1-critical_Y_ratio)

        green_signal_nb  = round(abs( Y_ratio/critical_Y_ratio *(Total_cycle_length-loss_time_per_cycle)))

        red_signal_nb = Total_cycle_length - green_signal_nb - Amber
    

        return green_signal_nb


def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB',RGB)

video1 =cv2.VideoCapture(0)
video2 =cv2.VideoCapture(1)
video3 = cv2.VideoCapture(2)
my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")
print(class_list)
count=0
tracker = Tracker()

car_entering = {}

count_car1 = set()
count_car2 = set()
count_car3 = set()


area3 = [(979,332),(605,270),(623,249),(986,301)]
area2 = [(979,332),(605,270),(623,249),(986,301)]
area1 = [(979,332),(545,240),(573,219),(986,301)]
while True:
    
    ret0,frame0 = video1.read()
    ret1,frame1 = video2.read()
    ret2,frame2 = video3.read()

    if not ret0:
        break
    count += 1
    if count % 3 != 0:
        continue
    frame0=cv2.resize(frame0,(1020,500))
    frame1=cv2.resize(frame1,(1020,500))
    frame2 = cv2.resize(frame2,(1020,500))
    

    results1=model.predict(frame0)
    results2 = model.predict(frame1)
    results3 = model.predict(frame2)

    a=results1[0].boxes.data
    b=results2[0].boxes.data
    c=results3[0].boxes.data
    px = pd.DataFrame(a).astype('float')
    py = pd.DataFrame(b).astype('float')
    pz = pd.DataFrame(c).astype('float')
    list1 = []
    list2 = []
    list3 = []
    #webcam1
    for index,row in px.iterrows():
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        '''if 'car' in c:
            result = cv2.pointPolygonTest(np.array(area,np.int32),((x1,y2)),False)
            if result >= 0:
                cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
                cv2.circle(frame,(x1,y2),4,(2555,0,0),-1)
                cv2.putText(frame,str(c),(x1,y1),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))'''
        if 'car' in c:
           list1.append([x1,y1,x2,y2])
    bbox_id = tracker.update(list1)
    for bbox in bbox_id:
        x3,y3,x4,y4,id = bbox
        result=cv2.pointPolygonTest(np.array(area1,np.int32),((x4,y4)),False)
        if result >= 0:
            car_entering[id]=(x4,y4)
            cv2.rectangle(frame0,(x3,y3),(x4,y4),(0,255,0),2)
            cv2.circle(frame0,(x4,y4),5,(2555,0,0),-1)
            cv2.putText(frame0,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))
            count_car1.add(id)
    number = len(count_car1) 
    cv2.putText(frame0,('Number of Cars: ')+ str(number),(60,40),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    cv2.putText(frame0,('Green Light =')+ str(signal_counter(number))+('s'),(60,80),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
 
    #webcam2
    for index,row in py.iterrows():
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        '''if 'car' in c:
            result = cv2.pointPolygonTest(np.array(area,np.int32),((x1,y2)),False)
            if result >= 0:
                cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
                cv2.circle(frame,(x1,y2),4,(2555,0,0),-1)
                cv2.putText(frame,str(c),(x1,y1),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))'''
        if 'car' in c:
           list2.append([x1,y1,x2,y2])
    cbox_id = tracker.update(list2)
    for bbox in cbox_id:
        x3,y3,x4,y4,id = bbox
        result=cv2.pointPolygonTest(np.array(area2,np.int32),((x4,y4)),False)
        if result >= 0:
            car_entering[id]=(x4,y4)
            cv2.rectangle(frame1,(x3,y3),(x4,y4),(0,255,0),2)
            cv2.circle(frame1,(x4,y4),5,(2555,0,0),-1)
            cv2.putText(frame1,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))
            count_car2.add(id) 
    number = len(count_car2) 
    cv2.putText(frame1,('Number of Cars: ')+ str(number),(60,40),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    cv2.putText(frame1,('Green Light =')+ str(signal_counter(number))+('s'),(60,80),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    #webcam3
    for index,row in pz.iterrows():
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        '''if 'car' in c:
            result = cv2.pointPolygonTest(np.array(area,np.int32),((x1,y2)),False)
            if result >= 0:
                cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
                cv2.circle(frame,(x1,y2),4,(2555,0,0),-1)
                cv2.putText(frame,str(c),(x1,y1),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))'''
        if 'car' in c:
           list3.append([x1,y1,x2,y2])
    dbox_id = tracker.update(list3)
    for bbox in dbox_id:
        x3,y3,x4,y4,id = bbox
        result=cv2.pointPolygonTest(np.array(area3,np.int32),((x4,y4)),False)
        if result >= 0:
            car_entering[id]=(x4,y4)
            cv2.rectangle(frame2,(x3,y3),(x4,y4),(0,255,0),2)
            cv2.circle(frame2,(x4,y4),5,(2555,0,0),-1)
            cv2.putText(frame2,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))
            count_car3.add(id) 
    number = len(count_car3) 
    cv2.putText(frame2,('Number of Cars: ')+ str(number),(60,40),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    cv2.putText(frame2,('Green Light =')+ str(signal_counter(number))+('s'),(60,80),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

        
    cv2.polylines(frame0,[np.array(area1,np.int32)],True,(255,0,255),2) 
    cv2.polylines(frame1,[np.array(area2,np.int32)],True,(255,0,255),2)
    cv2.polylines(frame2,[np.array(area3,np.int32)],True,(255,0,255),2)

    #print out number of cars
    ''' print('The number of cars from camera 1 is ' + str(len(count_car1)))
    print('Green Light = ' + str(signal_counter(count_car1)))    
    
    print('The number of cars from camera 2 is ' + str(len(count_car2)))
    print('Green Light = ' + str(signal_counter(count_car1)))
    
    print('The number of cars from camera 3 is ' + str(len(count_car3)))
    print('Green Light = ' + str(signal_counter(count_car1)))'''

    if (ret0):
        cv2.imshow('Camera 0',frame0)

    if (ret1):
        cv2.imshow('Camera 1',frame1)

    if (ret2):
        cv2.imshow('Camera 2',frame2)

    if cv2.waitKey(1)&0xFF== ord('q'):
        break




video1.release()
video2.release()
video3.release()
cv2.destroyAllWindows()

number_of_cars_lane1 = len(count_car1)
number_of_cars_lane2 = len(count_car2)
number_of_cars_lane3 = len(count_car3)


































['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [ ]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import *


model=YOLO('yolov8s.pt')


def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        


cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB',RGB)

cap=cv2.VideoCapture("Traffic IP Camera video (1).mp4")

my_file = open("coco.txt", "r")
data = my_file.read()   
class_list = data.split("\n")
print(class_list)
count=0
tracker = Tracker()

car_entering = {}
count_car = []
area = [(986,340),(587,260),(673,161),(1018,239)]

def signal_counter(counted_car):
    if counted_car == 0:
        pass
    else:
        number_of_cars = counted_car
        mean_time_headway = 30/number_of_cars
        flow_rate = 3600/mean_time_headway
        Amber = 2
 

        approach_width = 4

        saturation_flow = 525*approach_width

        Y_ratio = saturation_flow/flow_rate


        critical_Y_ratio = Y_ratio

        number_of_phases = 2

        time_loss = 2

        all_red = 4

        loss_time_per_cycle = number_of_phases*time_loss + all_red


        Total_cycle_length = (1.5*loss_time_per_cycle + 5)/(1-critical_Y_ratio)

        green_signal_nb  = round(abs( Y_ratio/critical_Y_ratio *(Total_cycle_length-loss_time_per_cycle)))

        red_signal_nb = Total_cycle_length - green_signal_nb - Amber

        return green_signal_nb

while True:

    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue
    frame=cv2.resize(frame,(1020,500))
    

    results=model.predict(frame)
    a=results[0].boxes.data
    px = pd.DataFrame(a).astype('float')
    list = []
    for index,row in px.iterrows():
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        '''if 'car' in c:
            result = cv2.pointPolygonTest(np.array(area,np.int32),((x1,y2)),False)
            if result >= 0:
                cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
                cv2.circle(frame,(x1,y2),4,(2555,0,0),-1)
                cv2.putText(frame,str(c),(x1,y1),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))'''
        if 'car' in c:
           list.append([x1,y1,x2,y2])
    bbox_id = tracker.update(list)
    for bbox in bbox_id:
        x3,y3,x4,y4,id = bbox
        result=cv2.pointPolygonTest(np.array(area,np.int32),((x4,y4)),False)
        if result >= 0:
            car_entering[id]=(x4,y4)
            cv2.rectangle(frame,(x3,y3),(x4,y4),(0,255,0),2)
            cv2.circle(frame,(x4,y4),5,(2555,0,0),-1)
            cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))
            if count_car.count(id)==0:
                count_car.append(id)
        
    cv2.polylines(frame,[np.array(area,np.int32)],True,(255,0,255),2)
    print(count_car)
    number = len(count_car) 
    cv2.putText(frame,('Number of Cars: ')+ str(number),(60,40),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    cv2.putText(frame,('Green Light =')+ str(signal_counter(number))+('s'),(60,80),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    print('The number of cars from camera 1 is ' + str(len(count_car)))
    print('Green Light = ' + str(signal_counter(number)))    
    cv2.imshow("RGB", frame)
    if cv2.waitKey(1)&0xFF== ord('q'):
        break

    

cap.release()
cv2.destroyAllWindows()


In [ ]:
model.val(split=px)

In [ ]:
#signal counter
number_of_cars = len(count_car)
mean_time_headway = 30/(number_of_cars)
flow_rate = 3600/mean_time_headway
Amber = 2

approach_width = 4

saturation_flow = 525*approach_width

Y_ratio = saturation_flow/flow_rate


critical_Y_ratio = Y_ratio

number_of_phases = 2

time_loss = 2

all_red = 4

loss_time_per_cycle = number_of_phases*time_loss + all_red


Total_cycle_length = (1.5*loss_time_per_cycle + 5)/(1-critical_Y_ratio)

green_signal_nb  = round(abs( Y_ratio/critical_Y_ratio *(Total_cycle_length-loss_time_per_cycle)))

red_signal_nb = abs(Total_cycle_length - green_signal_nb - Amber)

   
print(mean_time_headway,number_of_cars,green_signal_nb,red_signal_nb)


In [ ]:
int_to_string = str(green_signal_nb)
print(int_to_string)

In [ ]:
%pip install pyserial

In [ ]:
import serial

ser = serial.Serial(port='COM10',baudrate=9600)

while True:
    value = ser.write(bytes(int_to_string,'utf-8'))
    print(value)


In [ ]:
result_str = str(result)
start_index = result_str.find('cars')
end_index = result_str.find(',', start_index)
num_cars = int(result_str[start_index - 3:start_index - 1])
print(start_index)


In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*

model=YOLO('yolov8s.pt')



def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap=cv2.VideoCapture(0)


my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n") 
#print(class_list)

count=0

tracker=Tracker()
vh = {}
counter = []
cy1=322
cy2=368
offset=6

while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue
    frame=cv2.resize(frame,(1020,500))
   

    results=model.predict(frame)
 #   print(results)
    a=results[0].boxes.data
    px=pd.DataFrame(a).astype("float")
#    print(px)
    list=[]
             
    for index,row in px.iterrows():
#        print(row)
 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            list.append([x1,y1,x2,y2])
    bbox_id=tracker.update(list)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2

        if cy1 < (cy+offset) and cy1>(cy-offset):
            vh[id]=cy
        if id in vh:
            if cy1 < (cy+offset) and cy1>(cy-offset):
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
                if counter.count(id)==0:
                    counter.append(id)
           


    cv2.line(frame,(274,cy1),(814,cy1),(255,255,255),1)
    cv2.line(frame,(177,cy2),(927,cy2),(255,255,255),1)
    number = len(counter)
    cv2.putText(frame,('Number of Cars: ')+ str(number),(60,40),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    cv2.imshow("RGB", frame)
    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()